In [2]:
#import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import config

In [3]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [4]:
beer_db = pd.read_sql_query('''SELECT * FROM beer_data;''', con=engine)
beer_db.head()

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA


In [23]:
# Seperate dataset into features (X) and target (y)
y = beer_db["style"]
X = beer_db.drop(columns="style")

# splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(beer_db[['ibu','abv']], beer_db[['style']], random_state=48)

# Creating scaler instance

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train

,style
1103,Bock
80,Gose
414,American IPA
844,American Stout
612,American Strong Ale
...,...
966,American Pale Ale (APA)
944,Hefeweizen
347,American IPA
1361,American IPA


In [26]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=128, random_state=48).fit(X_train_scaled,(np.ravel(y_train)))

y_preds = rfc.predict(X_test_scaled)
print(y_preds)

['Saison / Farmhouse Ale' 'American Pale Ale (APA)' 'American Porter'
 'Saison / Farmhouse Ale' 'Fruit / Vegetable Beer' 'American Black Ale'
 'American IPA' 'American Pale Ale (APA)' 'American Strong Ale'
 'American Pale Ale (APA)' 'American IPA' 'American Pale Ale (APA)'
 'Czech Pilsener' 'American Brown Ale' 'American Brown Ale'
 'American Brown Ale' 'American IPA' 'American Double / Imperial IPA'
 'American Pale Ale (APA)' 'American IPA' 'American Double / Imperial IPA'
 'American IPA' 'Fruit / Vegetable Beer' 'American Blonde Ale'
 'American Double / Imperial IPA' 'Fruit / Vegetable Beer' 'Dunkelweizen'
 'American IPA' 'American IPA' 'American IPA' 'American Amber / Red Ale'
 'American Pale Wheat Ale' 'American IPA' 'Hefeweizen' 'Hefeweizen'
 'American IPA' 'American Pale Ale (APA)' 'American Brown Ale'
 'American Blonde Ale' 'American IPA' 'American Pilsner'
 'American Pale Ale (APA)' 'American Pale Wheat Ale' 'American IPA'
 'Fruit / Vegetable Beer' 'American Pale Wheat Ale' 'Ab

In [30]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_preds)

# Create a DataFrame from the confusion matrix.
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm
# print(classification_report(y_test,y_preds))
# print(accuracy_score(y_test, y_preds))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)